In [1]:
import pandas as pd
import numpy as np
import os
import datetime

os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/Rewards_NonR_LawGardenPatio_JT_20190215'

In [2]:
def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)

In [3]:
taxonomy=pd.read_csv("/home/jian/BigLots/static_files/ProductTaxonomy/MediaStormProductTaxonomy20190201-133832-059.txt",dtype=str,sep="|")
taxonomy['subclass_id']=taxonomy['subclass_id'].apply(lambda x: x.zfill(3))
division_id_id_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Division Names.txt",dtype=str,sep="|")
department_id_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Department Names.txt",dtype=str,sep="|")
class_id_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Class Names.txt",dtype=str,sep="|",encoding ='ISO-8859-1')

In [4]:
Patio_Furniture_ClassID=class_id_name[class_id_name['class_code_desc']=="PATIO FURNITURE"]['class_code_id'].unique().tolist()
if len(Patio_Furniture_ClassID)==1:
    Patio_Furniture_ClassID=Patio_Furniture_ClassID[0]
else:
    Patio_Furniture_ClassID=np.nan

Patio_RUGS_ClassID=class_id_name[class_id_name['class_code_desc']=="PATIO RUGS"]['class_code_id'].unique().tolist()
if len(Patio_RUGS_ClassID)==1:
    Patio_RUGS_ClassID=Patio_RUGS_ClassID[0]
else:
    Patio_RUGS_ClassID=np.nan


In [5]:
Law_and_Garden_DepartmentID=department_id_name[department_id_name['department_desc']=="LAWN & GARDEN"]['department_id'].tolist()
if len(Law_and_Garden_DepartmentID)==1:
    Law_and_Garden_DepartmentID=Law_and_Garden_DepartmentID[0]
else:
    Law_and_Garden_DepartmentID=np.nan
df_taxonomy_LawGarden=taxonomy[taxonomy['department_id']==Law_and_Garden_DepartmentID]
LawnGarden_ClassID_list=df_taxonomy_LawGarden['class_code_id'].unique().tolist()
set([len(x) for x in LawnGarden_ClassID_list])

{5}

In [6]:
all_2018_week_endings=[datetime.date(2019,2,2) - datetime.timedelta(days=x*7) for x in range(52)]
all_2018_week_endings_str=[str(x) for x in all_2018_week_endings]

In [7]:
historical_daily_files=list(recursive_file_gen("/home/jian/BigLots/hist_daily_data/"))
df_all_hist=pd.DataFrame()

for week_ends in all_2018_week_endings_str:
    for file in historical_daily_files:
        if week_ends in file:
            df=pd.DataFrame({"Path":file,"date":week_ends},index=[0])
            df_all_hist=df_all_hist.append(df)
df_all_hist=df_all_hist.sort_values(['date']).reset_index()
del df_all_hist['index']

In [8]:
weekly_delivered_2018=list(recursive_file_gen("/home/jian/BigLots/2018_by_weeks/"))
df_2018_files=pd.DataFrame()
for file in weekly_delivered_2018:
    if "aily" in file:
        df=pd.DataFrame({"Path":file,"date":file.split("/")[5].split("_")[1]},index=[0])
        df_2018_files=df_2018_files.append(df)
df_2018_files=df_2018_files.sort_values(['date']).reset_index()
del df_2018_files['index']

In [9]:
weekly_delivered_2019=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
df_2019_files=pd.DataFrame()
for file in weekly_delivered_2019:
    if "aily" in file:
        df=pd.DataFrame({"Path":file,"date":file.split("/")[5].split("_")[1]},index=[0])
        df_2019_files=df_2019_files.append(df)
df_2019_files=df_2019_files.sort_values(['date']).reset_index()
del df_2019_files['index']

In [10]:
all_daily_files=df_all_hist.append(df_2018_files).append(df_2019_files)
all_daily_files=all_daily_files[all_daily_files['Path'].apply(lambda x: ".txt" in x)]
all_daily_files=all_daily_files[all_daily_files['date'].isin(all_2018_week_endings_str)].sort_values("date").reset_index()
del all_daily_files['index']
all_daily_files.shape

(52, 2)

In [11]:
def get_quarter(x):
    if (x>="2018-02-10") and (x<="2018-05-05"):
        y="Q1"
    elif (x>="2018-05-12") and (x<="2018-08-04"):
        y="Q2"
    elif (x>="2018-08-11") and (x<="2018-11-03"):
        y="Q3"
    elif (x>="2018-11-10") and (x<="2019-02-02"):
        y="Q4"
    return y

all_daily_files['Quarter']=all_daily_files['date'].apply(lambda x: get_quarter(x))

In [12]:
def agg_df(dataframe):
    dataframe['subclass_transaction_amt']=dataframe['subclass_transaction_amt'].astype(float)
    dataframe['subclass_transaction_units']=dataframe['subclass_transaction_units'].astype(float)
    
    dataframe_rewards=dataframe[~pd.isnull(dataframe['customer_id_hashed'])]
    dataframe_NonRewards=dataframe[pd.isnull(dataframe['customer_id_hashed'])]
    
    dataframe_rewards_sales=dataframe_rewards.groupby(['location_id'])['subclass_transaction_amt','subclass_transaction_units'].sum().reset_index()
    dataframe_rewards_sales=dataframe_rewards_sales.rename(columns={"subclass_transaction_amt":"sales","subclass_transaction_units":"units"})
    dataframe_rewards_trans=dataframe_rewards[['location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
    dataframe_rewards_trans=dataframe_rewards_trans.groupby('location_id')['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"transactions"})
    df_rewards=pd.merge(dataframe_rewards_sales,dataframe_rewards_trans,on="location_id",how="outer")
    df_rewards['level']="Rewards"
    
    dataframe_NonRewards_sales=dataframe_NonRewards.groupby(['location_id'])['subclass_transaction_amt','subclass_transaction_units'].sum().reset_index()
    dataframe_NonRewards_sales=dataframe_NonRewards_sales.rename(columns={"subclass_transaction_amt":"sales","subclass_transaction_units":"units"})
    dataframe_NonRewards_trans=dataframe_NonRewards[['location_id','transaction_dt','transaction_id']].drop_duplicates()
    dataframe_NonRewards_trans=dataframe_NonRewards_trans.groupby('location_id')['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"transactions"})
    df_NonRewards=pd.merge(dataframe_NonRewards_sales,dataframe_NonRewards_trans,on="location_id",how="outer")
    df_NonRewards['level']="Non_Rewards"
    
    output_df=df_rewards.append(df_NonRewards)
    return output_df
    
    

In [13]:
df_LawGarden_all=pd.DataFrame()
df_Patio_Furniture_all=pd.DataFrame()
df_Patio_RUGS_all=pd.DataFrame()

counter_i=0

for i in range(len(all_daily_files)):
    date=all_daily_files['date'][i]
    file_path=all_daily_files['Path'][i]
    quarter=all_daily_files['Quarter'][i]
    
    df=pd.read_table(file_path,dtype=str,sep="|")
    df_LawGarden=df[df['class_code_id'].isin(LawnGarden_ClassID_list)]
    df_Patio_Furniture=df[df['class_code_id']==Patio_Furniture_ClassID]
    df_Patio_RUGS=df[df['class_code_id']==Patio_RUGS_ClassID]
    
    df_LawGarden=agg_df(df_LawGarden)
    df_Patio_Furniture=agg_df(df_Patio_Furniture)
    df_Patio_RUGS=agg_df(df_Patio_RUGS)
    
    df_LawGarden['week_end_dt']=date
    df_Patio_Furniture['week_end_dt']=date
    df_Patio_RUGS['week_end_dt']=date
    
    df_LawGarden['Quarter']=quarter
    df_Patio_Furniture['Quarter']=quarter
    df_Patio_RUGS['Quarter']=quarter
    
    df_LawGarden_all=df_LawGarden_all.append(df_LawGarden)
    df_Patio_Furniture_all=df_Patio_Furniture_all.append(df_Patio_Furniture)
    df_Patio_RUGS_all=df_Patio_RUGS_all.append(df_Patio_RUGS)
    
    counter_i+=1
    if counter_i%10==2:
        print(datetime.datetime.now())

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


2019-02-15 15:20:20.669796
2019-02-15 15:29:37.821130
2019-02-15 16:00:52.461194
2019-02-15 16:14:01.744224
2019-02-15 16:36:28.230559
2019-02-15 16:56:59.319768


In [14]:
df_LawGarden_all.shape

(137689, 7)

In [15]:
df_Patio_Furniture_all.shape

(117060, 7)

In [16]:
df_Patio_RUGS_all.shape

(58270, 7)

In [17]:
df_LawGarden_all['department_desc']="LAWN & GARDEN"

df_Patio_Furniture_all['class_code_desc']="PATIO FURNITURE"
df_Patio_RUGS_all['class_code_desc']="PATIO RUGS"

In [21]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/BL_Subset_LawnGarden_Patio_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_LawGarden_all.to_excel(writer,"Lawn_and_Garden",index=False)
df_Patio_Furniture_all.to_excel(writer,"PATIO FURNITURE",index=False)
df_Patio_RUGS_all.to_excel(writer,"PATIO RUGS",index=False)
writer.save()


In [19]:
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/Rewards_NonR_LawGardenPatio_JT_20190215'